https://www.youtube.com/watch?v=sZdIybqppqQ

In [ ]:
!pip install ktrain

In [ ]:
!git clone https://github.com/laxmimerit/IMDB-Movie-Reviews-Large-Dataset-50k.git

Cloning into 'IMDB-Movie-Reviews-Large-Dataset-50k'...
remote: Enumerating objects: 10, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 10 (delta 1), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (10/10), done.


In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import ktrain
from ktrain import text

In [ ]:
data_train = pd.read_excel('/content/IMDB-Movie-Reviews-Large-Dataset-50k/train.xlsx',dtype=str)
data_train.head()

,Reviews,Sentiment
0,"When I first tuned in on this morning news, I ...",neg
1,"Mere thoughts of ""Going Overboard"" (aka ""Babes...",neg
2,Why does this movie fall WELL below standards?...,neg
3,Wow and I thought that any Steven Segal movie ...,neg
4,"The story is seen before, but that does'n matt...",neg


In [ ]:
data_test = pd.read_excel('/content/IMDB-Movie-Reviews-Large-Dataset-50k/test.xlsx',dtype=str)
data_test.head()

,Reviews,Sentiment
0,Who would have thought that a movie about a ma...,pos
1,After realizing what is going on around us ......,pos
2,I grew up watching the original Disney Cindere...,neg
3,David Mamet wrote the screenplay and made his ...,pos
4,"Admittedly, I didn't have high expectations of...",neg


In [ ]:
(x_train,y_train),(x_test,y_test),preprocess=text.texts_from_df(train_df=data_train,text_column='Reviews',label_columns='Sentiment',
                   val_df=data_test,maxlen=380,
                   preprocess_mode = 'bert')

downloading pretrained BERT model (uncased_L-12_H-768_A-12.zip)...
[██████████████████████████████████████████████████]
extracting pretrained BERT model...
done.

cleanup downloaded zip...
done.

preprocessing train...
language: en


Is Multi-Label? False
preprocessing test...
language: en


In [ ]:
x_train[0].shape

(25000, 380)

In [ ]:
model = text.text_classifier(name='bert',train_data=(x_train,y_train),preproc=preprocess)

Is Multi-Label? False
maxlen is 380
done.


In [ ]:
# Get Learning Rate
learner = ktrain.get_learner(model=model,train_data=(x_train,y_train),val_data=(x_test,y_test),
                             batch_size=6)

In [ ]:
# might take days to run
# learner.lr_find(max_epochs=5)
# learner.lr_plot()
# got lr from here and input to the next step
# could use 5e-5, 3e-5, or 2e-5 for test purpose

In [ ]:
learner.fit_onecycle(lr=2e-5,epochs=1)



begin training using onecycle policy with max lr of 2e-05...
 301/4167 [=>............................] - ETA: 29:59:23 - loss: 0.5818 - accuracy: 0.7010

In [ ]:
predictor = ktrain.get_predictor(learner.model,preprocess)

In [ ]:
data=['this movie was horrible.The plot was really boring. Acting was okay. though',
      'the film really sucked. there is no plot and acting was bad',
      'what a beautiful movie. great plot, great acting, will see it again']

In [ ]:
predictor.predict(data)

In [ ]:
predictor.save('/content/bert')